<a href="https://colab.research.google.com/github/edponce/DoyleInvestigators/blob/master/Project1_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import re
import urllib.request
import urllib.parse

###$\color{brown}{\rm Corpus~Selection}$

In [3]:
CORPUS_URL = {
    'The Valley of Fear': "http://www.gutenberg.org/files/3289/3289.txt",
    'A Study of Scarlet': "http://www.gutenberg.org/files/244/244.txt",
    'The Sign of the Four': "http://www.gutenberg.org/files/2097/2097.txt",
    'The Hound of the Baskervilles': "http://www.gutenberg.org/files/2852/2852.txt",
}

###$\color{brown}{\rm Read~Web~Page~Content}$
Read the corpus from web page to start processing. Use text in ASCII format (no BOMs) and remove Windows-based newlines '\r'.

In [4]:
def get_corpus_from_url(url):
    with urllib.request.urlopen(url) as fd:
        return fd.read().decode('ascii').replace('\r', '')


def get_corpus_from_file(file):
    with open(file) as fd:
        return fd.read().replace('\r', '')


def get_corpus(key):
    def validate_url(url):
        parsed_url = urllib.parse.urlparse(url)
        return all([parsed_url.scheme, parsed_url.netloc, parsed_url.path])

    # Check if a filename was provided
    if os.path.isfile(key):
        return get_corpus_from_file(key)
    else:
        file = os.path.basename(CORPUS_URL.get(key))
        if os.path.isfile(file):
            return get_corpus_from_file(file)

    # Check if a URL was provided
    if validate_url(key):
        return get_corpus_from_url(key)
    else:
        url = CORPUS_URL.get(key)
        if validate_url(url):
            return get_corpus_from_url(url)

###$\color{brown}{\rm Split~into~Parts~and~Chapters}$
Simple regex expressions.

In [5]:
def get_gutenberg_start_tag(text):
    """Find Gutenberg's start tag (and producer, if available).

    Notes:
        * re.match() searches at the beginning of strings, but there are
          certain character combinations that are not considered strings,
          and thus need to use re.search(), even if it is at the beginning
          of line. An example are the asterisks in the Gutenberg START
          tag.
    """
    return re.search(r'\s+\*{3} START (\w+ )+\*{3}\s+(Produced by.+\s+)?', text)


def get_gutenberg_end_tag(text):
    """Find Gutenberg's end tag (and transcriber's notes, if available).

    Notes:
        * Duplicate/similar Gutenberg end tags.
        * Use a newline before transcriber note to prevent matching similar
          (but indented) notes at beginning of text.
        * Use DOTALL flag to match footnotes across multiple lines.
          Be wary of other regex parts that use dot operator, for example,
          change '.+' to '[^\n]+'.
    """
    return re.search(r'((\s+\nOriginal transcriber.*)?\s+End[^\n]+)?\s+\*{3} END (\w+ )+\*{3}\s+', text, flags=re.I|re.DOTALL)


def get_gutenberg_parts(text):
    """
    Notes:
        * We consider the start of the text when the first part/chapter starts.
        * Arabic or Roman numbering.
    """
    return list(re.finditer(r'\n+(Part (\d+|[IVX]+)(--|\.)?.*)\s+', text, flags=re.I))


def get_gutenberg_chapters(text):
    """
    Notes:
        * Arabic or Roman numbering.
        * Some text have the chapter tag and title in different lines.
    """
    return list(re.finditer(r'\n+(Chapter (\d+|[IVX]+)(--|\.)?(.+|\s+.+))\s+', text, flags=re.I))

In [6]:
def preprocess_text(key):
    corpus = get_corpus(key)

    parts = get_gutenberg_parts(corpus)
    print(len(parts))
    for part in parts:
       print(part.group(1))
       print(part.span())

    chapters = get_gutenberg_chapters(corpus)
    print(len(chapters))
    for chapter in chapters:
       print(chapter.group(1))
       print(chapter.span())

    start = get_gutenberg_start_tag(corpus)
    print(start)

    end = get_gutenberg_end_tag(corpus)
    print(end)

    return corpus

In [7]:
for title in CORPUS_URL:
    _ = preprocess_text(title)

2
Part 1--The Tragedy of Birlstone
(686, 728)
Part 2--The Scowrers
(150452, 150483)
14
Chapter 1--The Warning
(723, 754)
Chapter 2--Sherlock Holmes Discourses
(17533, 17579)
Chapter 3--The Tragedy of Birlstone
(32684, 32728)
Chapter 4--Darkness
(51304, 51332)
Chapter 5--The People of the Drama
(72051, 72094)
Chapter 6--A Dawning Light
(93981, 94016)
Chapter 7--The Solution
(119331, 119363)
Chapter 1--The Man
(150478, 150505)
Chapter 2--The Bodymaster
(167072, 167106)
Chapter 3--Lodge 341, Vermissa
(196539, 196578)
Chapter 4--The Valley of Fear
(226826, 226864)
Chapter 5--The Darkest Hour
(246306, 246342)
Chapter 6--Danger
(271488, 271514)
Chapter 7--The Trapping of Birdy Edwards
(289423, 289472)
<_sre.SRE_Match object; span=(539, 641), match='\n\n*** START OF THIS PROJECT GUTENBERG EBOOK THE>
<_sre.SRE_Match object; span=(312693, 312837), match="\n\n\n\n\n\nEnd of Project Gutenberg's The Valley>
2
PART I.
(1770, 1785)
PART II. _The Country of the Saints._
(122627, 122675)
14
CHAPTER I.

In [ ]:
#structure description:
'''
dict_sections = {
                    "part_1": {
                        "title":"The Tragedy of Birlstone",
                        "chapter_titles":['The Warning\r', 'Sherlock Holmes Discourses\r', 'The Tragedy of Birlstone\r', ...],
                        "chapters":["I believe that I am one of th...","It was one of those dramati...", ...]
                    },

                    "part_2": {
                        "title":"The Scowrers",
                        "chapter_titles":['The Warning\r', 'Sherlock Holmes Discourses\r', 'The Tragedy of Birlstone\r', ...],
                        "chapters":["I believe that I am one of th...","It was one of those dramati...", ...]
                    },
                    
                    ...,

                    "part_n":{
                        "title":"xyz",
                        "chapter_titles":['...', '...', ...],
                        "chapters":["...", "...", ...]
                    }
                }
'''

#dictionary containing the main parts of the novel, every part contains a list of chapters
dict_sections = {}


#Get the trimmed text of the novel
#--------------------------
#flags=re.I -> ignore case
start_novel_position = re.search('Part \d-', main_text, flags=re.I).span()[0]
end_novel_position = re.search('End of Project Gutenberg', main_text, flags=re.I).span()[0]

#trim text
main_text_trimmed = main_text[start_novel_position:end_novel_position].strip()
#--------------------------


#Get the parts of the novel
#--------------------------
list_text_parts = re.split("Part \d--", main_text_trimmed, flags=re.I) 

#the first item on the list is empty, this line gets rid of the empty item
list_text_parts = [part for part in list_text_parts if len(part) > 0] 
#--------------------------


#Get the chapters 
#--------------------------
#iterate over the 'Parts' to get chapters
for part_index, part in enumerate(list_text_parts):

    #get 'Part' title and 'Part' text
    linefeed_title = re.search('\n', part).span()[0]
    part_title = part[0:linefeed_title]
    part_text = part[linefeed_title:len(part)].strip()
    
    #get chapters list
    list_chapters = re.split("Chapter \d--", part_text, flags=re.I)
    list_chapters = [chapter for chapter in list_chapters if len(chapter) > 0] 

    #iterate over every chapter
    title_list = []
    chapter_list = []
    for chapter in list_chapters:
        #get 'Chapter' title and 'Chapter' text
        linefeed_title_ch = re.search('\n', chapter).span()[0]
        chapter_title = chapter[0:linefeed_title_ch]
        chapter_text = chapter[linefeed_title_ch:len(chapter)].strip()
        
        #add to lists
        title_list.append(chapter_title)
        chapter_list.append(chapter_text)

    #create a dictionary for this part
    part_dictionary = {"part_title":part_title,
                       "chapter_titles":title_list,
                       "chapters":chapter_list}

    #add to the main dictionary
    dict_sections["part_"+str(part_index+1)] = part_dictionary
#--------------------------


print(dict_sections["part_2"]["part_title"])

The Scowrers
